### <b> 전략 <b>
1. 가치주 : 일자 기준 저pbr, 저per, 저pcr 종목 (단, per>0이고 pbr>0인 종목)
2. 수익성 : eps 변화율>0인 종목을 찾습니다. 
3. 안정성 : 부채비율<200, 부채비율 변화율<0인 종목을 찾습니다.
3. 마지막으로 매수를 위해 이격도가 평균이동선 보다 낮은 값을 찾아 13개의 종목을 스크리닝 합니다.

In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import json
import requests

from pykrx import stock

In [108]:
#업데이트의 편의를 위해 api 사용
data = stock.get_market_cap("20240319", market="ALL")
data = data[['종가', '시가총액']]
data.head()

,종가,시가총액
티커,,
005930,72800,434600169640000
000660,160200,116625978873000
373220,403000,94302000000000
207940,848000,60355552000000
005935,61800,50854398060000


### 1.1 가치주 : 저per, 저pbr, 

In [109]:
#api 활용 데이터 - per, pbr
df = stock.get_market_fundamental_by_ticker(date='20240319', market="ALL")
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,947,106.33,3.37,30,0.00,0
095570,8076,24.50,0.61,201,5.48,270
006840,41948,0.00,0.37,0,1.29,200
054620,10864,14.57,0.68,505,0.00,0
265520,17980,3.72,1.13,5463,1.33,270


In [110]:
df.describe() #per<5 고정 #PBR의 경우 상위 25% 값보다 작은 것

,BPS,PER,PBR,EPS,DIV,DPS
count,2.622000e+03,2622.000000,2622.000000,2622.000000,2622.000000,2622.000000
mean,2.195963e+04,32.011850,2.345397,1718.613654,1.280561,343.768879
std,1.204684e+05,278.339978,11.258188,10138.308960,2.135117,1160.520751
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.452750e+03,0.000000,0.480000,0.000000,0.000000,0.000000
50%,4.738000e+03,5.700000,0.960000,143.000000,0.000000,0.000000
75%,1.258025e+04,17.295000,2.020000,938.500000,1.947500,200.000000
max,4.906107e+06,9342.860000,498.530000,406090.000000,37.100000,20000.000000


In [111]:
#업데이트 편의를 위해 넘파이 사용
import numpy as np
column_pbr = df['PBR'].values
pbr_25 = np.percentile(column_pbr, 25)

df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < pbr_25)] #저per, 저pbr인 데이터
df2 = df2[['PER', 'PBR']]
df2

,PER,PBR
티커,,
006840,0.00,0.37
138930,3.30,0.26
001465,0.00,0.00
035760,0.00,0.44
00104K,0.00,0.00
...,...,...
028080,0.00,0.45
079980,0.00,0.34
005010,0.89,0.31


In [112]:
df3 = pd.merge(data, df2, on='티커')
df3

,종가,시가총액,PER,PBR
티커,,,,
005935,61800,50854398060000,0.00,0.00
015760,22400,14379995324800,0.00,0.35
024110,14700,11722160274300,4.43,0.40
316140,14660,11023579098260,3.50,0.37
011200,15680,10804139297280,0.76,0.37
...,...,...,...,...
001527,11500,3548543500,0.00,0.00
021045,8300,3521241800,0.00,0.00
001525,5690,3514889390,0.00,0.00


### 1.2 PER > 0, PBR > 0

In [113]:
df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3 = df3[['종가', 'PER', 'PBR']]
df3.shape #per>0 이고 pbr>0인 65개의 항목을 추출합니다

(141, 3)

In [114]:
df3.index

Index(['024110', '316140', '011200', '078930', '004020', '088350', '138930',
       '036460', '000880', '139480',
       ...
       '000950', '012620', '050120', '225590', '054940', '377460', '001770',
       '073540', '106240', '115570'],
      dtype='object', name='티커', length=141)

### 1.3 저PCR
- pcr = 주가/(영업활동현금흐름/발행주식수)

In [115]:
#추가적인 정보를 가져오기 위해 requests 라이브러리를 활용하였습니다.
ticker = 290120
url = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd=290120&fin_typ=4&freq_typ=A&encparam=dHBSMjZyTzBlaDZGd2p2enFUeThsZz09&id=ZlEwemUxRm'
headers= {
'Referer':'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more',
'User-Agent' :
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
'X-Requested-With' :
'XMLHttpRequest' }

res = requests.get(url=url, headers=headers)
df = pd.read_html(res.text)[1]
df

주요재무정보        연간연간컨센서스보기                                      \
        주요재무정보 2020/12  (IFRS연결) 2021/12  (IFRS연결) 2022/12  (IFRS연결)   
0          매출액           1718.00           1790.00           2260.00   
1         영업이익            141.00            114.00            223.00   
2   영업이익(발표기준)            141.00            114.00            223.00   
3     세전계속사업이익             73.00            118.00            303.00   
4        당기순이익             58.00             82.00            239.00   
5    당기순이익(지배)             58.00             83.00            238.00   
6   당기순이익(비지배)              0.00             -1.00              0.00   
7         자산총계           1729.00           2182.00           2329.00   
8         부채총계           1229.00           1526.00           1445.00   
9         자본총계            501.00            656.00            884.00   
10    자본총계(지배)            500.00            654.00            879.00   
11   자본총계(비지배)              1.00              2.00              5.00   
12         자본금             54.00             57.00             59.00   
13    영업활동현금흐름            210.00             59.00            199.00   
14    투자활동현금흐름           -316.00           -118.00           -140.00   
15    재무활동현금흐름            233.00            253.00            -44.00   
16       CAPEX             81.00            137.00             96.00   
17         FCF            129.00            -78.00            103.00   
18      이자발생부채            781.00           1051.00            985.00   
19       영업이익률              8.21              6.36              9.88   
20        순이익률              3.36              4.61             10.56   
21      ROE(%)             11.93             14.39             31.10   
22      ROA(%)              3.83              4.22             10.58   
23        부채비율            245.41            232.47            163.47   
24       자본유보율            824.40            971.08           1339.56   
25      EPS(원)            530.00            743.00           2066.00   
26      PER(배)             14.10              8.02              2.61   
27      BPS(원)           4593.00           5723.00           7447.00   
28      PBR(배)              1.63              1.04              0.72   
29    현금DPS(원)            200.00            200.00            200.00   
30     현금배당수익률              2.68              3.36              3.71   
31   현금배당성향(%)             37.76             27.54              9.90   
32  발행주식수(보통주)       10884299.00       11433768.00       11798661.00   

                               분기분기컨센서스보기                                      \
   2023/12(E)  (IFRS연결) 2023/03  (IFRS연결) 2023/06  (IFRS연결) 2023/09  (IFRS연결)   
0                   NaN            584.00            603.00            557.00   
1                   NaN             52.00             27.00             64.00   
2                   NaN             52.00             27.00             64.00   
3                   NaN             40.00              5.00             57.00   
4                   NaN             33.00              2.00             44.00   
5                   NaN             33.00              1.00             44.00   
6                   NaN              0.00              1.00              0.00   
7                   NaN           2354.00           2515.00           2368.00   
8                   NaN           1437.00           1580.00           1408.00   
9                   NaN            918.00            935.00            959.00   
10                  NaN            912.00            929.00            954.00   
11                  NaN              5.00              6.00              6.00   
12                  NaN             60.00             62.00             64.00   
13                  NaN             59.00             47.00             30.00   
14                  NaN           -130.00            -23.00            -88.00   
15                  NaN             -1.00             89.00           -148.00   
16                  NaN      

In [116]:
df

주요재무정보        연간연간컨센서스보기                                      \
        주요재무정보 2020/12  (IFRS연결) 2021/12  (IFRS연결) 2022/12  (IFRS연결)   
0          매출액           1718.00           1790.00           2260.00   
1         영업이익            141.00            114.00            223.00   
2   영업이익(발표기준)            141.00            114.00            223.00   
3     세전계속사업이익             73.00            118.00            303.00   
4        당기순이익             58.00             82.00            239.00   
5    당기순이익(지배)             58.00             83.00            238.00   
6   당기순이익(비지배)              0.00             -1.00              0.00   
7         자산총계           1729.00           2182.00           2329.00   
8         부채총계           1229.00           1526.00           1445.00   
9         자본총계            501.00            656.00            884.00   
10    자본총계(지배)            500.00            654.00            879.00   
11   자본총계(비지배)              1.00              2.00              5.00   
12         자본금             54.00             57.00             59.00   
13    영업활동현금흐름            210.00             59.00            199.00   
14    투자활동현금흐름           -316.00           -118.00           -140.00   
15    재무활동현금흐름            233.00            253.00            -44.00   
16       CAPEX             81.00            137.00             96.00   
17         FCF            129.00            -78.00            103.00   
18      이자발생부채            781.00           1051.00            985.00   
19       영업이익률              8.21              6.36              9.88   
20        순이익률              3.36              4.61             10.56   
21      ROE(%)             11.93             14.39             31.10   
22      ROA(%)              3.83              4.22             10.58   
23        부채비율            245.41            232.47            163.47   
24       자본유보율            824.40            971.08           1339.56   
25      EPS(원)            530.00            743.00           2066.00   
26      PER(배)             14.10              8.02              2.61   
27      BPS(원)           4593.00           5723.00           7447.00   
28      PBR(배)              1.63              1.04              0.72   
29    현금DPS(원)            200.00            200.00            200.00   
30     현금배당수익률              2.68              3.36              3.71   
31   현금배당성향(%)             37.76             27.54              9.90   
32  발행주식수(보통주)       10884299.00       11433768.00       11798661.00   

                               분기분기컨센서스보기                                      \
   2023/12(E)  (IFRS연결) 2023/03  (IFRS연결) 2023/06  (IFRS연결) 2023/09  (IFRS연결)   
0                   NaN            584.00            603.00            557.00   
1                   NaN             52.00             27.00             64.00   
2                   NaN             52.00             27.00             64.00   
3                   NaN             40.00              5.00             57.00   
4                   NaN             33.00              2.00             44.00   
5                   NaN             33.00              1.00             44.00   
6                   NaN              0.00              1.00              0.00   
7                   NaN           2354.00           2515.00           2368.00   
8                   NaN           1437.00           1580.00           1408.00   
9                   NaN            918.00            935.00            959.00   
10                  NaN            912.00            929.00            954.00   
11                  NaN              5.00              6.00              6.00   
12                  NaN             60.00             62.00             64.00   
13                  NaN             59.00             47.00             30.00   
14                  NaN           -130.00            -23.00            -88.00   
15                  NaN             -1.00             89.00           -148.00   
16                  NaN      

In [117]:
import pandas as pd
import json
import requests

# 분석할 종목들
tickers = df3.index

# 결과를 저장할 데이터프레임
result_df = pd.DataFrame(columns=['종목', '변화율'])

# 각 종목에 대해 반복하여 데이터 수집 및 분석 수행
for ticker in tickers:
    # URL 및 Referer 생성
    url_template = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd={}&fin_typ=4&freq_typ=Y&encparam=dHBSMjZyTzBlaDZGd2p2enFUeThsZz09&id=ZlEwemUxRm'
    url = url_template.format(ticker)
    referer_template = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more'
    referer = referer_template.replace('290120', ticker)
    
    # HTTP 요청
    headers = {
        'Referer': referer,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }
    res = requests.get(url=url, headers=headers)
    df = pd.read_html(res.text)[1]
    
    # 데이터 전처리 및 분석
    df = df[[('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'), ('연간', '2022/12  (IFRS연결)')]]
    df = df.iloc[[13, 23, 24, 25, 32]]
    df['변화율'] = (df.iloc[:, 2] - df.iloc[:, 1]) / df.iloc[:, 1]
    
    # 종목별 결과를 데이터프레임에 추가
    df['종목'] = ticker   # '종목' 열에 티커 값 할당
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df

,종목,변화율,"(주요재무정보, 주요재무정보)","(연간, 2021/12 (IFRS연결))","(연간, 2022/12 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,영업활동현금흐름,-2.347000e+03,-1.469840e+05,61.626331,024110
1,NaN,NaN,부채비율,1.345750e+03,1.383930e+03,0.028371,024110
2,NaN,NaN,자본유보율,5.538000e+02,5.912400e+02,0.067606,024110
3,NaN,NaN,EPS(원),3.032000e+03,3.478000e+03,0.147098,024110
4,NaN,NaN,발행주식수(보통주),7.443006e+08,7.974259e+08,0.071376,024110
...,...,...,...,...,...,...,...
700,NaN,NaN,영업활동현금흐름,-6.000000e+01,1.530000e+02,-3.550000,115570
701,NaN,NaN,부채비율,6.905000e+01,4.903000e+01,-0.289935,115570
702,NaN,NaN,자본유보율,1.768670e+03,1.934490e+03,0.093754,115570
703,NaN,NaN,EPS(원),6.690000e+02,8.200000e+02,0.225710,115570


In [118]:
result_df.columns

Index([                       '종목',                       '변화율',
              ('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'),
       ('연간', '2022/12  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [119]:
grouped_df = result_df.groupby(('종목', ''))
grouped_df.head()

,종목,변화율,"(주요재무정보, 주요재무정보)","(연간, 2021/12 (IFRS연결))","(연간, 2022/12 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,영업활동현금흐름,-2.347000e+03,-1.469840e+05,61.626331,024110
1,NaN,NaN,부채비율,1.345750e+03,1.383930e+03,0.028371,024110
2,NaN,NaN,자본유보율,5.538000e+02,5.912400e+02,0.067606,024110
3,NaN,NaN,EPS(원),3.032000e+03,3.478000e+03,0.147098,024110
4,NaN,NaN,발행주식수(보통주),7.443006e+08,7.974259e+08,0.071376,024110
...,...,...,...,...,...,...,...
700,NaN,NaN,영업활동현금흐름,-6.000000e+01,1.530000e+02,-3.550000,115570
701,NaN,NaN,부채비율,6.905000e+01,4.903000e+01,-0.289935,115570
702,NaN,NaN,자본유보율,1.768670e+03,1.934490e+03,0.093754,115570
703,NaN,NaN,EPS(원),6.690000e+02,8.200000e+02,0.225710,115570


In [120]:
df = result_df.drop(columns=['종목', '변화율'])
df.columns

Index([       ('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'),
       ('연간', '2022/12  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [121]:
df = df.rename(columns={('주요재무정보', '주요재무정보'): '재무비율', 
                        ('연간', '2021/12  (IFRS연결)'): '2021/12', 
                        ('연간', '2022/12  (IFRS연결)'): '2022/12', 
                        ('변화율', ''): '변화율', 
                        ('종목', ''): '종목'})

In [122]:
df.columns

Index(['재무비율', '2021/12', '2022/12', '변화율', '종목'], dtype='object')

In [123]:
df

,재무비율,2021/12,2022/12,변화율,종목
0,영업활동현금흐름,-2.347000e+03,-1.469840e+05,61.626331,024110
1,부채비율,1.345750e+03,1.383930e+03,0.028371,024110
2,자본유보율,5.538000e+02,5.912400e+02,0.067606,024110
3,EPS(원),3.032000e+03,3.478000e+03,0.147098,024110
4,발행주식수(보통주),7.443006e+08,7.974259e+08,0.071376,024110
...,...,...,...,...,...
700,영업활동현금흐름,-6.000000e+01,1.530000e+02,-3.550000,115570
701,부채비율,6.905000e+01,4.903000e+01,-0.289935,115570
702,자본유보율,1.768670e+03,1.934490e+03,0.093754,115570
703,EPS(원),6.690000e+02,8.200000e+02,0.225710,115570


In [124]:
df['재무비율'].value_counts()

영업활동현금흐름      141
부채비율          141
자본유보율         141
EPS(원)        141
발행주식수(보통주)    141
Name: 재무비율, dtype: int64

In [129]:
#PCR 계산
ten1 = df.loc[(df['재무비율']=='영업활동현금흐름')][['2022/12', '종목']]
ten2 = df.loc[(df['재무비율']=='발행주식수(보통주)')][['2022/12', '종목']]
ten3 = stock.get_market_ohlcv('20221229', market='ALL')
merged_df = pd.merge(ten1, ten2, on='종목', how='outer', suffixes=('_영업활동현금흐름', '_발행주식수'))
ten3.index.name = '종목'
merged_df2 = pd.merge(merged_df, ten3, on='종목', how='outer')

merged_df2 = merged_df2.dropna()
pcr_ratio = merged_df2[['종목', '2022/12_영업활동현금흐름', '2022/12_발행주식수', '종가']]
pcr_ratio['PCR'] = pcr_ratio['종가'] / (pcr_ratio['2022/12_영업활동현금흐름']/pcr_ratio['2022/12_발행주식수'])
pcr_ratio['PCR'].describe()

count    1.180000e+02
mean     1.193511e+09
std      1.785315e+10
min     -7.500000e+10
25%     -5.062631e+07
50%      1.749494e+08
75%      4.956103e+08
max      1.664976e+11
Name: PCR, dtype: float64

In [131]:
pcr_ratio = pcr_ratio.loc[(pcr_ratio['PCR'] < 1.749494e+08)]

In [132]:
df = pd.merge(df, pcr_ratio, on='종목')
df

,재무비율,2021/12,2022/12,변화율,종목,2022/12_영업활동현금흐름_x,2022/12_발행주식수_x,종가_x,PCR_x,2022/12_영업활동현금흐름_y,2022/12_발행주식수_y,종가_y,PCR_y
0,영업활동현금흐름,-2.347000e+03,-1.469840e+05,61.626331,024110,-146984.0,797425869.0,9820.0,-5.327602e+07,-146984.0,797425869.0,9820,-5.327602e+07
1,부채비율,1.345750e+03,1.383930e+03,0.028371,024110,-146984.0,797425869.0,9820.0,-5.327602e+07,-146984.0,797425869.0,9820,-5.327602e+07
2,자본유보율,5.538000e+02,5.912400e+02,0.067606,024110,-146984.0,797425869.0,9820.0,-5.327602e+07,-146984.0,797425869.0,9820,-5.327602e+07
3,EPS(원),3.032000e+03,3.478000e+03,0.147098,024110,-146984.0,797425869.0,9820.0,-5.327602e+07,-146984.0,797425869.0,9820,-5.327602e+07
4,발행주식수(보통주),7.443006e+08,7.974259e+08,0.071376,024110,-146984.0,797425869.0,9820.0,-5.327602e+07,-146984.0,797425869.0,9820,-5.327602e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,영업활동현금흐름,4.650000e+02,2.710000e+02,-0.417204,106240,271.0,15923057.0,2010.0,1.181009e+08,271.0,15923057.0,2010,1.181009e+08
291,부채비율,1.099500e+02,5.690000e+01,-0.482492,106240,271.0,15923057.0,2010.0,1.181009e+08,271.0,15923057.0,2010,1.181009e+08
292,자본유보율,4.644800e+02,3.406220e+03,6.333405,106240,271.0,15923057.0,2010.0,1.181009e+08,271.0,15923057.0,2010,1.181009e+08
293,EPS(원),1.143000e+03,7.078000e+03,5.192476,106240,271.0,15923057.0,2010.0,1.181009e+08,271.0,15923057.0,2010,1.181009e+08


In [133]:
df['종목'].nunique() #118개의 종목 필터링

59

### 2. 부채비율, eps 변화율

In [134]:
#부채비율 <200이고, 부채비율이 전년도에 비해 감소한 기업
df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0) & (df['2022/12'] < 200)]['종목'].unique()
temp1 = df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0)]
temp1.shape 

(33, 13)

In [135]:
#EPS가 증가한 기업 필터링 
df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]['종목'].unique()
temp2 = df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]
temp2.shape 

(34, 13)

In [136]:
#저per이고, 부채비율이 감소하였으면서, eps가 증가한 기업 필터링
final = pd.merge(temp1, temp2, on='종목')
final['종목'] #재무비율 활용 끝

0     316140
1     011200
2     078930
3     047040
4     001120
5     003240
6     016380
7     032190
8     123890
9     151860
10    005010
11    007860
12    003200
13    002020
14    007160
15    005710
16    053700
17    004720
18    101330
19    140520
20    000950
21    050120
22    106240
Name: 종목, dtype: object

### 4. 매수 위해 이격도 확인

In [189]:
data = stock.get_market_ohlcv('20231201', '20240320', ticker)
data = data[['종가']]
std_dev = data['종가'].rolling(20).std() 

data['중심 밴드'] = data['종가'].rolling(20).mean()
data['이격도'] = (data['종가'] / data['중심 밴드']) * 100
data.dropna()

,종가,중심 밴드,이격도
날짜,,,
2024-01-02,1818,1734.35,104.823133
2024-01-03,1784,1736.70,102.723556
2024-01-04,1773,1739.00,101.955147
2024-01-05,1774,1742.45,101.810669
2024-01-08,1788,1748.20,102.276627
2024-01-09,1790,1754.00,102.052452
2024-01-10,1779,1759.70,101.096778
2024-01-11,1759,1762.20,99.818409
2024-01-12,1723,1760.70,97.858806


In [190]:
ticker_list = final['종목'].tolist()
ticker_list

['316140',
 '011200',
 '078930',
 '047040',
 '001120',
 '003240',
 '016380',
 '032190',
 '123890',
 '151860',
 '005010',
 '007860',
 '003200',
 '002020',
 '007160',
 '005710',
 '053700',
 '004720',
 '101330',
 '140520',
 '000950',
 '050120',
 '106240']

In [199]:
#이격도가 이동평균선보다 낮은 티커 필터링
#날짜에 따라 이격도가 다를 수 있음
result_df = pd.DataFrame()
tickers = ticker_list

for ticker in tickers : 
    data = stock.get_market_ohlcv('20230320', '20240320', ticker)
    data = data[['종가']]
    std_dev = data['종가'].rolling(20).std() 
    
    data['중심'] = data['종가'].rolling(20).mean() 
    data['이격도'] = (data['종가'] / data['중심']) * 100
    data['상단'] = data['중심'] + 2 * data['종가'].rolling(window=20).std()
    data['하단'] = data['중심'] - 2 * data['종가'].rolling(window=20).std()
    
    data = data.dropna()
    
    result_df[ticker] = data['이격도']

result_df = result_df.T

In [218]:
#이격도가 이동평균선보다 낮은 티커 필터링
#날짜에 따라 이격도가 다를 수 있음
result_df = pd.DataFrame()
tickers = ticker_list

for ticker in tickers : 
    data = stock.get_market_ohlcv('20200320', '20240320', ticker)
    data = data[['종가']]
    std_dev = data['종가'].rolling(20).std() 
    
    data['중심'] = data['종가'].rolling(20).mean() 
    data['상단'] = data['중심'] + 2 * data['종가'].rolling(window=20).std()
    data['하단'] = data['중심'] - 2 * data['종가'].rolling(window=20).std()
    data['이격도'] = (data['종가'] / data['중심']) * 100
    
    data = data.dropna()
    result_df[ticker] = data['이격도']

result_df = result_df.T

In [215]:
data

,종가,중심,상단,하단,이격도
날짜,,,,,
2020-04-17,1380,1067.80,1328.345662,807.254338,129.237685
2020-04-20,1337,1090.10,1362.984707,817.215293,122.649298
2020-04-21,1322,1113.70,1380.732109,846.667891,118.703421
2020-04-22,1524,1140.55,1457.302917,823.797083,133.619745
2020-04-23,1578,1169.65,1533.868139,805.431861,134.912153
...,...,...,...,...,...
2024-03-14,1464,1546.55,1709.488122,1383.611878,94.662313
2024-03-15,1483,1540.30,1702.902648,1377.697352,96.279945
2024-03-18,1500,1533.65,1691.086403,1376.213597,97.805888


In [210]:
data[['중심', '종가', '상단', '하단']].iplot(kind='scatter') #하락추세임을 확인

In [223]:
#20일 기준 이격도의 경우 매수 시점은 98 정도로 알려져있습니다.
#20240320 기준 
#거래일 종가가 하단선보다 아래에 있다면 매수하는 것을 반영하였습니다
final_df = result_df.loc[(result_df['2024-03-20'] < 98)] 
final_df.shape

(13, 968)

In [224]:
final_df.index

Index(['011200', '047040', '001120', '003240', '016380', '032190', '151860',
       '002020', '007160', '053700', '101330', '050120', '106240'],
      dtype='object')

### <b> 종목 정리 <b> 
    20240320 기준
    1. 011200 HMM 
    2. 047040 대우건설
    3. 001120 LX 인터내셔널
    4. 003240 태광산업
    5. 016380 KG스틸
    6. 032190 다우데이타
    7. 151860 KG ETS
    8. 002020 코오롱
    9. 007160 사조산업
    10. 053700 삼보모터스
    11. 101330 모베이스
    12. 050120 ES 큐브
    13. 106240 파인테크닉스